<a href="https://colab.research.google.com/github/vedanthamramaysree/Model_Demo_Assignment2/blob/vedanthamramaysree-patch-1/Huggingface_model_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install transformers needed for downloading hugging face model
!pip install --quiet transformers==4.6.1

# text visualization purpose
!pip install textwrap3==0.9.2

     |████████████████████████████████| 2.2 MB 28.1 MB/s 
     |████████████████████████████████| 880 kB 55.0 MB/s 
     |████████████████████████████████| 3.3 MB 50.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# check whether the device is cpu or GPU
import torch
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
print ("Device ", torch_device)


Device  cuda


In [ ]:
! nvcc --version 

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [ ]:
# download tokenizers
from transformers.pipelines import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

#download pre trained distil bart model used for summarization in our case
tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")
model = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-cnn-12-6")
model = model.to(torch_device)

Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

In [ ]:
# define a function for summarizing
def get_summary(t,tokenizer_summary,model_summary):
  txt = t['text']
  minl = t['min_length'] #75
  maxl = t['max_length'] #150
  inputs = tokenizer_summary([txt], max_length=1024,truncation=True, return_tensors='pt').to(torch_device)
  summary_ids = model_summary.generate(inputs['input_ids'], num_beams=3,num_return_sequences=1,no_repeat_ngram_size=2, min_length = minl,max_length=maxl, early_stopping=True)
  dec = [tokenizer_summary.decode(ids,skip_special_tokens=True, clean_up_tokenization_spaces=True) for ids in summary_ids]
  output = dec[0].strip()
  return {'summary':output}

In [ ]:
from textwrap3 import wrap

#sample text from an article to be summarized"
text = """
High-quality data allows companies and supply chain managers to improve operational efficiencies, monitor inventory management, provide analytics information on budget, timely procurement management, and much more.According to tech research firm Gartner, it is expected that by 2025, 80% of supply chain applications will use AI in practice. “The world’s gotten too complex to try to manage some of these things on spreadsheets,” said Dwight Klappich, a Gartner analyst.This article is about how we performed quality health checks on supply chain data and applied simple programming techniques to reduce manual data-checking processes by 40%.Accurate product data is a critical feature in supply chain systems as it is the foundation of supply chain transactions and aids in faster decision-making. A weak inaccurate data point would disrupt the entire system challenging stakeholders who tend to make unreliable conclusions.We found that to large extent the characteristics of data quality are highly dependent on the input health provided to the system. Also, it is impossible for humans to manually review huge data excel files and perform quality checks.As a team of data scientists and data engineers, we provided a solution to this challenge by conducting data health checks to detect areas of risk, and data discrepancies and identified data improvement opportunities, resulting in a more stable, secure environment and high-quality data in supply chain management.
"""
for s in wrap(text,150):
  print (s)

 High-quality data allows companies and supply chain managers to improve operational efficiencies, monitor inventory management, provide analytics
information on budget, timely procurement management, and much more.According to tech research firm Gartner, it is expected that by 2025, 80% of
supply chain applications will use AI in practice. “The world’s gotten too complex to try to manage some of these things on spreadsheets,” said Dwight
Klappich, a Gartner analyst.This article is about how we performed quality health checks on supply chain data and applied simple programming
techniques to reduce manual data-checking processes by 40%.Accurate product data is a critical feature in supply chain systems as it is the foundation
of supply chain transactions and aids in faster decision-making. A weak inaccurate data point would disrupt the entire system challenging stakeholders
who tend to make unreliable conclusions.We found that to large extent the characteristics of data quality are high

In [ ]:
#final output as a summary
payload = {'text':text,'min_length':75,'max_length':150}
summ = get_summary(payload,tokenizer,model)
summary = summ['summary']

for s in wrap(summary,150):
  print (s)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1818: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


By 2025, 80% of supply chain applications will use AI in practice, says Gartner. High-quality data allows companies to improve operational
efficiencies, monitor inventory management, provide analytics information on budget, timely procurement management and much more. It is impossible for
humans to manually review huge data excel files and perform quality checks. We found that to large extent the characteristics of data quality are
highly dependent on the input health provided to the system.
